In [ ]:
# Loaded variable 'df' from URI: c:\Users\GGUK\Desktop\Engineering_EKB\data\new_df_66.xlsx
import pandas as pd
df = pd.read_excel(r'c:\\Users\\GGUK\\Desktop\\Engineering_EKB\\data\\new_df_66.xlsx')


In [1]:
# Specify the path to the PDB file
pdb_file = r"c:\\Users\\GGUK\\Desktop\\Engineering_EKB\\data\\EKB1_model1_cys.pdb"

# Dictionary to map amino acids to single letter codes
amino_acid_dict = {
    'ALA': 'A',
    'ARG': 'R',
    'ASN': 'N',
    'ASP': 'D',
    'CYS': 'C',
    'GLN': 'Q',
    'GLU': 'E',
    'GLY': 'G',
    'HIS': 'H',
    'ILE': 'I',
    'LEU': 'L',
    'LYS': 'K',
    'MET': 'M',
    'PHE': 'F',
    'PRO': 'P',
    'SER': 'S',
    'THR': 'T',
    'TRP': 'W',
    'TYR': 'Y',
    'VAL': 'V'
}

residue_dict = {}

for line in open(pdb_file):
    if line.startswith('ATOM'):
        residue_number = line.split()[5]
        amino_acid = line.split()[3]
        single_letter_code = amino_acid_dict.get(amino_acid, '')
        residue_dict[residue_number] = single_letter_code

residue_word = ''.join(residue_dict.values())
import os
# Define the folder path
folder_path = "c:\\Users\\GGUK\\Desktop\\Engineering_EKB\\data\\FASTA"

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Define the file path
file_path = os.path.join(folder_path, "wild_type.fasta")

# Write the sequence to the file
with open(file_path, "w") as file:
    file.write(">wild_type\n")
    file.write(residue_word)

residue_word[174]

'Y'

In [64]:
import pandas as pd
import os

# Load the data
df = pd.read_excel(r'c:\\Users\\GGUK\\Desktop\\Engineering_EKB\\data\\CleanVariants.xlsx')

# Define the folder path for output
folder_path = "c:\\Users\\GGUK\\Desktop\\Engineering_EKB\\data\\FASTA"

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    # Get the ID and clean variants for the current row
    ID = row['ID']
    clean_variants = row['Clean Variants'].split(',')  # Split the variants string into a list

    # Initialize mutated_sequence with the original sequence
    # Ensure that `residue_word` is correctly defined before this loop
    mutated_sequence = residue_word

    # Clean and process each variant
    for variant in clean_variants:
        try:
            variant = variant.strip("[] '")  # Remove brackets, spaces, and single quotes
            position = int(variant[1:-1])  # Extract the numeric position
            mutation = variant[-1]  # Extract the mutation character

            # Apply the mutation to the sequence
            mutated_sequence = mutated_sequence[:position-1] + mutation + mutated_sequence[position:]
        except ValueError:
            # If a ValueError occurs, print the ID and the problematic variant, then continue
            print(f"Skipping variant due to error: {variant} in ID: {ID}")
            continue

    # Define the file path for the mutated FASTA file
    file_path = os.path.join(folder_path, f"{ID}.fasta")

    # Write the mutated sequence to the file
    with open(file_path, "w") as file:
        file.write(f">{ID}\n")
        file.write(mutated_sequence)


Skipping variant due to error: "H242'F" in ID: WLEK0673
Skipping variant due to error: "H246'Q" in ID: WLEK0673
Skipping variant due to error:  in ID: WLEK0833


In [104]:
# Define a function to transform the string
def transform_string(s):
    # Remove brackets and quotes, then add "B " prefix
    return "B " + s.replace("['", "").replace("']", "").replace("', '", ";B ")

# Apply the function to the 'Original' column
df['DDM'] = df['Clean Variants'].apply(transform_string)
df
df.to_excel(r'c:/Users/GGUK/Desktop/Engineering_EKB/data/MDD.xlsx', index=False)


In [122]:
import re
import csv

# Define the pattern for a single mutation
# This pattern matches a single uppercase letter or digit for the chain ID,
# followed by a single uppercase letter for the wild-type amino acid,
# followed by one or more digits for the residue position,
# followed by a single uppercase letter for the mutant amino acid.
mutation_pattern = re.compile(r'^[A-Z0-9] [A-Z][0-9]+[A-Z]$')

def check_mutation_format(mutation):
    """
    Check if a single mutation matches the expected format.
    """
    return bool(mutation_pattern.match(mutation))

def check_line_format(line):
    """
    Check if all mutations in a line match the expected format.
    """
    mutations = line.split(';')
    return all(check_mutation_format(mutation.strip()) for mutation in mutations)

def validate_file(filename):
    """
    Validate each line in the given CSV file.
    """
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        line_number = 1
        for row in reader:
            # Assuming each row has a single column with the mutation list
            if not check_line_format(row[0]):
                print(f"Line {line_number} does not match the expected format: {row[0]}")
            else:
                print(f"Line {line_number} is correctly formatted.")
            line_number += 1

# Replace 'your_file.csv' with the path to your actual CSV file
validate_file(r'C:\Users\GGUK\Desktop\Engineering_EKB\data\DDM.csv')


Line 1 is correctly formatted.
Line 2 is correctly formatted.
Line 3 is correctly formatted.
Line 4 is correctly formatted.
Line 5 is correctly formatted.
Line 6 is correctly formatted.
Line 7 is correctly formatted.
Line 8 is correctly formatted.
Line 9 is correctly formatted.
Line 10 is correctly formatted.
Line 11 is correctly formatted.
Line 12 is correctly formatted.
Line 13 is correctly formatted.
Line 14 is correctly formatted.
Line 15 is correctly formatted.
Line 16 is correctly formatted.
Line 17 is correctly formatted.
Line 18 is correctly formatted.
Line 19 is correctly formatted.
Line 20 is correctly formatted.
Line 21 is correctly formatted.
Line 22 is correctly formatted.
Line 23 is correctly formatted.
Line 24 is correctly formatted.
Line 25 is correctly formatted.
Line 26 is correctly formatted.
Line 27 is correctly formatted.
Line 28 is correctly formatted.
Line 29 is correctly formatted.
Line 30 is correctly formatted.
Line 31 is correctly formatted.
Line 32 is correc

In [18]:
from Bio import Align
from Bio.Align import substitution_matrices
import glob
import numpy as np
import pandas as pd

# Load the BLOSUM62 substitution matrix
blosum62 = substitution_matrices.load("BLOSUM62")

# Create a PairwiseAligner object
aligner = Align.PairwiseAligner()

# Set the substitution matrix to BLOSUM62
aligner.substitution_matrix = blosum62

# Define the path to the wild type FASTA file
wild_type_file = r"C:\Users\GGUK\Desktop\Engineering_EKB\data\FASTA\wild_type.fasta"

# Define the path to the protein variants FASTA files
all_fasta_files = glob.glob(r"C:\Users\GGUK\Desktop\Engineering_EKB\data\FASTA\*.fasta")

variant_files = [file for file in all_fasta_files if file != wild_type_file]

# Function to read sequence from a FASTA file
def read_sequence_from_file(file_path):
    with open(file_path, "r") as file:
        return ''.join(file.read().splitlines()[1:])  # Join all lines except the first (header)

# Read the wild type sequence
wild_type_sequence = read_sequence_from_file(wild_type_file)

# Define the path to the CleanVariants Excel file
clean_variants_file = r'c:/Users/GGUK/Desktop/Engineering_EKB/data/CleanVariants.xlsx'

# Read the CleanVariants Excel file
df_clean_variants = pd.read_excel(clean_variants_file)

# Extract the ID from the variant file name
def extract_id_from_filename(filename):
    print(os.path.splitext(os.path.basename(filename))[0])
    # Extracts the base name of the file without its path or extension
    return os.path.splitext(os.path.basename(filename))[0]


# Add a new column "BLOSUM62 Score" to the dataframe
df_clean_variants['BLOSUM62 Score'] = np.nan

# Process each variant file
for variant_file in variant_files:
    variant_sequence = read_sequence_from_file(variant_file)
    
    # Perform alignment
    alignments = aligner.align(wild_type_sequence, variant_sequence)
    best_alignment = alignments[0]  # Assuming the first alignment is the best
    
    # Calculate the total score
    total_score = best_alignment.score
    
    # Extract the ID from the variant file name
    variant_id = extract_id_from_filename(variant_file)
    
    # Update the "BLOSUM62 Score" column for the corresponding ID
    df_clean_variants.loc[df_clean_variants['ID'] == variant_id, 'BLOSUM62 Score'] = total_score

# Save the updated dataframe to the CleanVariants Excel file
df_clean_variants.to_excel(clean_variants_file, index=False)


WLEK0294
WLEK0296
WLEK0297
WLEK0298
WLEK0303
WLEK0305
WLEK0306
WLEK0309
WLEK0316
WLEK0321
WLEK0322
WLEK0323
WLEK0324
WLEK0327
WLEK0328
WLEK0330
WLEK0333
WLEK0334
WLEK0335
WLEK0342
WLEK0343
WLEK0346
WLEK0347
WLEK0348
WLEK0349
WLEK0350
WLEK0358
WLEK0362
WLEK0369
WLEK0389
WLEK0391
WLEK0393
WLEK0394
WLEK0396
WLEK0397
WLEK0399
WLEK0403
WLEK0404
WLEK0405
WLEK0406
WLEK0408
WLEK0409
WLEK0410
WLEK0411
WLEK0414
WLEK0415
WLEK0418
WLEK0419
WLEK0420
WLEK0421
WLEK0422
WLEK0423
WLEK0430
WLEK0433
WLEK0435
WLEK0440
WLEK0443
WLEK0444
WLEK0445
WLEK0446
WLEK0447
WLEK0448
WLEK0450
WLEK0452
WLEK0453
WLEK0454
WLEK0456
WLEK0457
WLEK0458
WLEK0460
WLEK0463
WLEK0466
WLEK0468
WLEK0470
WLEK0472
WLEK0473
WLEK0474
WLEK0477
WLEK0482
WLEK0485
WLEK0486
WLEK0487
WLEK0488
WLEK0490
WLEK0491
WLEK0492
WLEK0494
WLEK0495
WLEK0496
WLEK0498
WLEK0499
WLEK0500
WLEK0501
WLEK0502
WLEK0505
WLEK0507
WLEK0508
WLEK0509
WLEK0511
WLEK0512
WLEK0513
WLEK0514
WLEK0515
WLEK0517
WLEK0518
WLEK0520
WLEK0521
WLEK0522
WLEK0523
WLEK0524
WLEK0526
W